## Runner

This notebook parallelizes many experiments.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %matplotlib widget

In [ ]:
from __future__ import annotations

from concurrent import futures

import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
from tqdm.auto import tqdm

import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

from run import run

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
def run_wrapper(seed, desc=None, **kwargs):
    print(' ', end='', flush=True)
    pbar = tqdm(position=seed)
    if desc is not None:
        pbar.set_description(desc)
    else:
        pbar.set_description(f'seed: {seed}')
    run(pbar=pbar, seed=seed, **kwargs)

In [ ]:
with futures.ProcessPoolExecutor(max_workers=4) as pool:
    DEFAULT_KWARGS = dict(
        epsilon=0.1,
        q_max=0.24,
        eta=8.65,
        is_interactive=False,
        savedir='out')

    future_to_seed = {}

    # no-noise baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=0.2, noise=0)
    # seed = 0
    # future = pool.submit(run_wrapper, seed=seed, **DEFAULT_KWARGS, **kwargs)
    # future_to_seed[future] = seed

    # low-noise baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=0.2, noise=0.4)
    # for seed in range(4):
    #     future = pool.submit(run_wrapper, seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # low-noise + perm baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=0.2, noise=0.4, perm=True)
    # for seed in [14]:
    #     future = pool.submit(run_wrapper, desc=f'low-noise baseline (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # kwargs = dict(cbc_alg='proj', norm_bound=0.2, noise=0.4, perm=True)
    # for seed in [14]:
    #     future = pool.submit(run_wrapper, desc=f'low-noise (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # med-noise baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=0.2, noise=0.8)
    # for seed in range(4, 8):
    #     future = pool.submit(run_wrapper, seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # med-noise + perm baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=0.5, noise=0.5, perm=True)
    # for seed in [55, 56, 57, 58]:
    #     future = pool.submit(run_wrapper, desc=f'med-noise baseline (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # kwargs = dict(cbc_alg='proj', norm_bound=0.5, noise=0.5, perm=True)
    # for seed in [55, 56, 57, 58]:
    #     future = pool.submit(run_wrapper, desc=f'med-noise (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # kwargs = dict(cbc_alg='const', norm_bound=0.6, noise=0.6, perm=True)
    # for seed in [66, 67]:
    #     future = pool.submit(run_wrapper, desc=f'med-noise baseline (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # high-noise + perm baseline (no CBC)
    # kwargs = dict(cbc_alg='const', norm_bound=1.0, noise=1.0, perm=True)
    # for seed in range(8, 10):
    #     future = pool.submit(run_wrapper, seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # high-noise + perm
    # kwargs = dict(cbc_alg='proj', norm_bound=1.0, noise=1.0, perm=True)
    # for seed in [8, 9, 10, 11]:
    #     future = pool.submit(run_wrapper, desc=f'high-noise (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed

    # high-noise + perm (partial-obs)
    all_nodes = np.arange(55)
    exclude = np.array([9, 19, 22, 31, 40, 46, 55]) - 2
    obs_nodes = np.setdiff1d(all_nodes, exclude).tolist()
    kwargs = dict(cbc_alg='proj', norm_bound=1.0, noise=1.0, modify='perm',
                  obs_nodes=obs_nodes, tag='_partialobs')
    for seed in [8, 9, 10, 11]:
        future = pool.submit(run_wrapper, desc=f'high-noise + perm + partial-obs (seed {seed})',
                             seed=seed, **DEFAULT_KWARGS, **kwargs)
        future_to_seed[future] = seed
    
    # large uncertainty set, with med-noise + perm
    # kwargs = dict(cbc_alg='proj', norm_bound=1.0, noise=0.5, norm_bound_init=0.5, perm=True)
    # for seed in [55, 56, 57, 58]:
    #     future = pool.submit(run_wrapper, tag=f'large uncertainty / med-noise (seed {seed})',
    #                          seed=seed, **DEFAULT_KWARGS, **kwargs)
    #     future_to_seed[future] = seed
    
    print('all submitted')
    for future in futures.as_completed(future_to_seed):
        seed = future_to_seed[future]
        try:
            save_path = future.result()
        except Exception as exc:
            print('seed %r generated an exception: %s' % (seed, exc))